In [1]:
import csv
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import geopy.distance
import math
import string
import researchpy as rp
from statsmodels.formula.api import ols

import scipy.stats as stats


In [2]:


pred_procesising_df = pd.read_csv("busi_alc_fight_df3.csv", index_col=0)

reasonable_df = pd.read_csv("reasonable_crime.csv", engine = "python")
reasonable_df = reasonable_df.set_index("stop_id")

In [3]:
reasonable_df = reasonable_df.loc[~reasonable_df.index.duplicated(keep='first')]



In [4]:
business_df = pred_procesising_df.copy(deep=True)

In [5]:
business_df.drop("lat", axis=1, inplace = True)
business_df.drop("lon", axis=1, inplace = True)
business_df.rename({"ACTIVITY DESC" : "ACTIVITY_DESC"},axis=1, inplace = True)



In [6]:
business_df.head()


,ACTIVITY_DESC,2447,2455,2485,2532,2577,2578,2603,2681,2708,...,83256,83284,83318,83333,83335,83336,83359,83365,83366,83369
0,INFORMATION SERVICES,5.931638,10.826005,12.152264,2.963084,14.701244,5.165864,5.061442,6.966756,2.434433,...,4.872376,5.262967,10.865340,2.703858,5.096691,9.812076,2.731830,3.500604,2.727225,0.629066
1,ALL OTHER HEALTH & PERSONAL CARE STORES,2.392861,7.206127,8.598698,3.528617,18.441642,1.577430,2.160008,3.316914,3.580974,...,1.129519,1.554527,7.252293,3.374546,1.443737,6.128407,3.270481,5.091399,3.276868,3.174340
2,REAL ESTATE,2.214189,5.652816,7.630572,4.199080,19.989504,0.208213,1.116841,2.557939,4.494928,...,1.086112,0.578970,5.694998,4.162429,0.415001,4.988487,4.044222,5.770857,4.052822,4.897985
3,OTHER INDIVIDUAL & FAMILY SERVICES,2.239962,7.251192,8.483424,3.803150,18.474400,1.753749,2.458503,3.219919,3.815571,...,1.148897,1.651299,7.299620,3.633638,1.592499,6.056638,3.533317,5.351134,3.539266,3.163615
4,INVESTIGATION SERVICES,5.748061,3.089895,2.089211,10.899886,26.358912,6.745512,7.572055,4.663792,11.175719,...,6.776582,6.492860,3.156492,10.856802,6.726134,1.808533,10.738622,12.477863,10.747115,11.016866


In [7]:
alc_fight_df = reasonable_df.loc[reasonable_df["crime_type"].isin(["*fight / battery*", "*alcohol*"])]



In [8]:
def renameCrimeType(id) :
    if(str(id).isnumeric()):
        try :
            return alc_fight_df.loc[int(id)]["crime_type"][1:6] #id + "_" + 
        except :
            pass
    return id


business_df.rename(columns = lambda x: renameCrimeType(x), inplace = True)
business_df.head()


,ACTIVITY_DESC,fight,fight,alcoh,fight,alcoh,fight,alcoh,fight,fight,...,fight,alcoh,alcoh,fight,alcoh,fight,fight,fight,alcoh,fight
0,INFORMATION SERVICES,5.931638,10.826005,12.152264,2.963084,14.701244,5.165864,5.061442,6.966756,2.434433,...,4.872376,5.262967,10.865340,2.703858,5.096691,9.812076,2.731830,3.500604,2.727225,0.629066
1,ALL OTHER HEALTH & PERSONAL CARE STORES,2.392861,7.206127,8.598698,3.528617,18.441642,1.577430,2.160008,3.316914,3.580974,...,1.129519,1.554527,7.252293,3.374546,1.443737,6.128407,3.270481,5.091399,3.276868,3.174340
2,REAL ESTATE,2.214189,5.652816,7.630572,4.199080,19.989504,0.208213,1.116841,2.557939,4.494928,...,1.086112,0.578970,5.694998,4.162429,0.415001,4.988487,4.044222,5.770857,4.052822,4.897985
3,OTHER INDIVIDUAL & FAMILY SERVICES,2.239962,7.251192,8.483424,3.803150,18.474400,1.753749,2.458503,3.219919,3.815571,...,1.148897,1.651299,7.299620,3.633638,1.592499,6.056638,3.533317,5.351134,3.539266,3.163615
4,INVESTIGATION SERVICES,5.748061,3.089895,2.089211,10.899886,26.358912,6.745512,7.572055,4.663792,11.175719,...,6.776582,6.492860,3.156492,10.856802,6.726134,1.808533,10.738622,12.477863,10.747115,11.016866


In [9]:

business_df.set_index("ACTIVITY_DESC", inplace = True)


In [10]:
business_df = business_df.groupby(business_df.index).filter(lambda x : len(x)>99)
business_type_count = business_df.index.value_counts()

In [11]:

#weights_df = (1 / business_df) * (1 / business_df)
#weights_df

In [ ]:
maxDistance = 20 #km
floor_df = maxDistance / business_df

floor_df = np.floor(floor_df)
floor_df = floor_df / floor_df
floor_df

In [13]:
#floor_df = floor_df.groupby(floor_df.index).sum()


Divide by the sample size to get avrage number of of each type businesses for each crime

In [ ]:
#floor_df = (floor_df.T / business_type_count).T


In [ ]:
'''
fight_df = floor_df.filter(like='fight')
fight_summery_df = rp.summary_cont(fight_df.T)
alcoh_df = floor_df.filter(like='alcoh')
alcoh_summery_df = rp.summary_cont(alcoh_df.T)
'''



In [ ]:
fight_summery_df

In [ ]:
#fight_df["business"] = fight_df.index

In [ ]:
fight_df

In [ ]:
results = ols('fight ~ C(ACTIVITY_DESC)', data=alcoh_df).fit()


In [17]:
fight_df = floor_df.filter(like='fight')
